<h1>Snow Clearance Fines, 2019-2023</h1>
31 January 2024

This analysis looks at fines levied for uncleared sidewalks, based on FOIA data requested from the Department of Administrative Hearings (H064920-011124.xlsx). This dataset contained 3058 records dating from 1/1/2001 to 9/12/2023; filtered for those between 7/1/2019 and 6/30/2023 has 2560 records. Four of these could not be geocoded due to "unknown" address.<br>
<br>
My analysis steps:
<ol>
<li><a href="#read">Read Data</a>
    <li><a href="#summarize">Summarize</a>- by issuing department, by year, by community area
        <li><a href="#community">Community Summary</a>
</ol>

### Preliminary Findings
<ul>
    <li>Englewood, Garfield Ridge, and West Englewood are the three communities with the highest number of dockets per capita
    <li>17 of 25 police-related dockets related to snow clearance were in Englewood, West Englewood, Garfield Ridge, and Belmont Cragin
    </ul>

<a name="read"></a>
# 1. Read Data

In [1]:
import pandas as pd
import requests
import numpy as np
import altair as alt
#import datetime as dt #would only need this if I manipulated dates post-API data retrieval

In [2]:
df = pd.read_csv("../data/fines-geocoded-w-communities.csv")
df.head()

,field_1,Docket Number,Violation Date,Violation Address,Issuing Department Code,Imposed Fine Detailed,year,month,date,season,...,latlong,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len
0,416,20DT000917,2020/02/14,3100 S INDIANA,TRANPORT,0.0,2020,2,2020/02/14,2019-2020,...,"41.838241,-87.622033",DOUGLAS,0.0,4.600462e+07,0.0,35.0,35.0,0.0,0.0,31027.05451
1,417,20DT000917,2020/02/14,3100 S INDIANA,TRANPORT,150.0,2020,2,2020/02/14,2019-2020,...,"41.838241,-87.622033",DOUGLAS,0.0,4.600462e+07,0.0,35.0,35.0,0.0,0.0,31027.05451
2,418,20DT000917,2020/02/14,3100 S INDIANA,TRANPORT,500.0,2020,2,2020/02/14,2019-2020,...,"41.838241,-87.622033",DOUGLAS,0.0,4.600462e+07,0.0,35.0,35.0,0.0,0.0,31027.05451
3,757,21DT000478,2021/01/28,3317 S PRAIRIE,TRANPORT,110.0,2021,1,2021/01/28,2020-2021,...,"41.834323092878925,-87.62050356526285",DOUGLAS,0.0,4.600462e+07,0.0,35.0,35.0,0.0,0.0,31027.05451
4,773,21DT000493,2021/01/29,3658 S PRAIRIE,TRANPORT,500.0,2021,1,2021/01/29,2020-2021,...,"41.828145583678925,-87.62059581323",DOUGLAS,0.0,4.600462e+07,0.0,35.0,35.0,0.0,0.0,31027.05451


In [3]:
len(df)

2557

<a name="summarize"></a>
# 2. Summarize Citywide

### by season

In [4]:
# by season
df.groupby('season').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    max_fine=('Imposed Fine Detailed', 'max'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()

,season,sum_fine_amt,max_fine,n_dockets,n_records
0,2019-2020,96790.0,1200.0,357,526
1,2020-2021,243210.0,1000.0,762,994
2,2021-2022,214199.0,5000.0,698,913
3,2022-2023,31570.0,2500.0,96,124


### by issuing department

In [5]:
# by issuing department
df.groupby('Issuing Department Code').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()

,Issuing Department Code,sum_fine_amt,n_dockets,n_records
0,BAFCONP,0.0,2,2
1,POLICE,1700.0,25,30
2,STRTSAN,176610.0,497,659
3,TRANPORT,407459.0,1389,1866


### by Address

In [6]:
df_by_address = df.groupby(['Violation Address','community']).agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()
df_by_address[df_by_address['n_dockets']>=3].sort_values("n_dockets",ascending=False).head()

,Violation Address,community,sum_fine_amt,n_dockets,n_records
714,3110 W 61ST ST,CHICAGO LAWN,0.0,6,6
1717,932 W 59TH ST,ENGLEWOOD,8500.0,4,10
1422,627 W RANDOLPH ST,NEAR WEST SIDE,1300.0,4,5
1054,4710 S WESTERN AVE,BRIGHTON PARK,1400.0,4,6
1034,4601 S ROCKWELL ST,BRIGHTON PARK,1000.0,3,4


### distribution of fines by year

In [7]:
bins = [0, 1, 150, 500, 501, 5000]
labels = ['0', '1-150', '151-499', '500', '501-5000']

df['Binned Fine'] = pd.cut(df['Imposed Fine Detailed'], bins=bins, labels=labels, right=False, include_lowest=True)

pivot_table = df.pivot_table(
    values='Imposed Fine Detailed',  # We sum over the original fine amounts
    index='Binned Fine',  # Use the binned column as the new index
    columns='season',  # Using 'season' for the columns
    aggfunc='count',  # Summing up the fine amounts
    fill_value=0  # Replace NaN with 0
)

pivot_table

season,2019-2020,2020-2021,2021-2022,2022-2023
Binned Fine,,,,
0,159,186,168,27
1-150,50,140,95,14
151-499,200,303,382,41
500,108,355,244,38
501-5000,9,10,22,4


In [8]:
pivot_table_long = pivot_table.reset_index().melt(id_vars='Binned Fine', var_name='Season', value_name='Count')

# Create the histogram (bar chart) with Altair
chart = alt.Chart(pivot_table_long).mark_bar().encode(
    x=alt.X('Binned Fine:N', title='Fine Amount Bins', sort=labels),  # Ensure custom order
    y=alt.Y('sum(Count):Q', title='Count'),
    color='Season:N',
    column='Season:N'  # Separate charts for each season
).properties(
    width=220,
    height=200
).resolve_scale(
    y='independent'  # This allows each chart to have a separate Y-axis scale
).properties(
    width=220,
    height=200
)

chart.display()

/home/robcat26/.local/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

<a name="community"></a>
# 3. Summarize by Community

### by community

In [9]:
df_community = df.groupby('community').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()
df_community.head()

,community,sum_fine_amt,n_dockets,n_records
0,ALBANY PARK,3360.0,15,18
1,ARCHER HEIGHTS,3580.0,12,15
2,ARMOUR SQUARE,3990.0,26,33
3,ASHBURN,200.0,2,2
4,AUBURN GRESHAM,9250.0,19,29


In [10]:
df_community.dtypes

community        object
sum_fine_amt    float64
n_dockets         int64
n_records         int64
dtype: object

### read community population

In [11]:
# retrieved on 1/11/24, but 2020 Census Population figures should be static
df_population = pd.read_csv("../data/population_cmap_2022.csv")

In [12]:
# simplify dataframe to get only essentials
df_population = df_population[['GEOID','GEOG','2020_POP']]
df_population = df_population.rename(columns={'GEOG':'COMMUNITY_NAME'})
df_population['COMMUNITY_CAPS']=df_population['COMMUNITY_NAME'].str.upper()
df_population.head()

,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,14,Albany Park,48396,ALBANY PARK
1,57,Archer Heights,14196,ARCHER HEIGHTS
2,34,Armour Square,13890,ARMOUR SQUARE
3,70,Ashburn,41098,ASHBURN
4,71,Auburn Gresham,44878,AUBURN GRESHAM


In [13]:
df_population.dtypes

GEOID              int64
COMMUNITY_NAME    object
2020_POP           int64
COMMUNITY_CAPS    object
dtype: object

### merge in community population data

In [14]:
df_community_summary = pd.merge(df_community,df_population,left_on='community',right_on='COMMUNITY_CAPS')
df_community_summary.head()

,community,sum_fine_amt,n_dockets,n_records,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,ALBANY PARK,3360.0,15,18,14,Albany Park,48396,ALBANY PARK
1,ARCHER HEIGHTS,3580.0,12,15,57,Archer Heights,14196,ARCHER HEIGHTS
2,ARMOUR SQUARE,3990.0,26,33,34,Armour Square,13890,ARMOUR SQUARE
3,ASHBURN,200.0,2,2,70,Ashburn,41098,ASHBURN
4,AUBURN GRESHAM,9250.0,19,29,71,Auburn Gresham,44878,AUBURN GRESHAM


In [15]:
# per 10,000 capita, per year over 4 years
df_community_summary['dockets per 10k'] = \
(10000/4)*df_community_summary['n_dockets']/df_community_summary['2020_POP']

In [16]:
df_community_summary

,community,sum_fine_amt,n_dockets,n_records,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dockets per 10k
0,ALBANY PARK,3360.0,15,18,14,Albany Park,48396,ALBANY PARK,0.774857
1,ARCHER HEIGHTS,3580.0,12,15,57,Archer Heights,14196,ARCHER HEIGHTS,2.113271
2,ARMOUR SQUARE,3990.0,26,33,34,Armour Square,13890,ARMOUR SQUARE,4.679626
3,ASHBURN,200.0,2,2,70,Ashburn,41098,ASHBURN,0.121660
4,AUBURN GRESHAM,9250.0,19,29,71,Auburn Gresham,44878,AUBURN GRESHAM,1.058425
...,...,...,...,...,...,...,...,...,...
65,WEST LAWN,5750.0,17,23,65,West Lawn,33662,WEST LAWN,1.262551
66,WEST PULLMAN,1450.0,5,9,53,West Pullman,26104,WEST PULLMAN,0.478854
67,WEST RIDGE,4800.0,24,28,2,West Ridge,77122,WEST RIDGE,0.777988
68,WEST TOWN,27565.0,97,127,24,West Town,87781,WEST TOWN,2.762557


In [17]:
df_community_summary.sort_values('dockets per 10k',ascending=False).head(10)

,community,sum_fine_amt,n_dockets,n_records,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dockets per 10k
20,ENGLEWOOD,44960.0,69,107,68,Englewood,24369,ENGLEWOOD,7.078666
24,GARFIELD RIDGE,21980.0,91,122,56,Garfield Ridge,35439,GARFIELD RIDGE,6.419481
63,WEST ENGLEWOOD,27130.0,74,102,67,West Englewood,29647,WEST ENGLEWOOD,6.240092
25,GRAND BOULEVARD,16770.0,52,71,38,Grand Boulevard,24589,GRAND BOULEVARD,5.286917
50,OAKLAND,4170.0,13,19,36,Oakland,6799,OAKLAND,4.780115
17,EAST GARFIELD PARK,11460.0,38,54,27,East Garfield Park,19992,EAST GARFIELD PARK,4.751901
2,ARMOUR SQUARE,3990.0,26,33,34,Armour Square,13890,ARMOUR SQUARE,4.679626
35,LINCOLN PARK,40739.0,128,168,7,Lincoln Park,70492,LINCOLN PARK,4.539522
10,BRIGHTON PARK,22340.0,81,105,58,Brighton Park,45053,BRIGHTON PARK,4.494706
33,KENWOOD,9430.0,33,46,39,Kenwood,19116,KENWOOD,4.315756


In [18]:
df_community_summary.to_csv("../results/admin-hearings-by-community.csv")

# 4. Police-Issued Fines

In [19]:
df_police = df[df['Issuing Department Code']=='POLICE']
df_police

,field_1,Docket Number,Violation Date,Violation Address,Issuing Department Code,Imposed Fine Detailed,year,month,date,season,...,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,Binned Fine
232,1522,22CP001080,2022/01/16,6135 S VERNON AVE,POLICE,0.0,2022,1,2022/01/16,2021-2022,...,WOODLAWN,0.0,5.781518e+07,0.0,42.0,42.0,0.0,0.0,46936.959244,0
233,1527,22CP001691,2022/01/29,6331 S RHODES,POLICE,0.0,2022,1,2022/01/29,2021-2022,...,WOODLAWN,0.0,5.781518e+07,0.0,42.0,42.0,0.0,0.0,46936.959244,0
411,49,20CP00166A,2020/02/13,5959 W GRAND AVE,POLICE,0.0,2020,2,2020/02/13,2019-2020,...,BELMONT CRAGIN,0.0,1.090994e+08,0.0,19.0,19.0,0.0,0.0,43311.707379,0
412,50,20CP00166A,2020/02/13,5959 W GRAND AVE,POLICE,150.0,2020,2,2020/02/13,2019-2020,...,BELMONT CRAGIN,0.0,1.090994e+08,0.0,19.0,19.0,0.0,0.0,43311.707379,151-499
413,51,20CP00167A,2020/02/13,5940 W FULLERTON AVE,POLICE,0.0,2020,2,2020/02/13,2019-2020,...,BELMONT CRAGIN,0.0,1.090994e+08,0.0,19.0,19.0,0.0,0.0,43311.707379,0
414,59,20CP012097,2020/02/28,6111 W DICKENS,POLICE,0.0,2020,2,2020/02/28,2019-2020,...,BELMONT CRAGIN,0.0,1.090994e+08,0.0,19.0,19.0,0.0,0.0,43311.707379,0
415,60,20CP012098,2020/02/28,6117 W DICKENS,POLICE,0.0,2020,2,2020/02/28,2019-2020,...,BELMONT CRAGIN,0.0,1.090994e+08,0.0,19.0,19.0,0.0,0.0,43311.707379,0
416,61,20CP012099,2020/02/28,6159 W DICKENS,POLICE,0.0,2020,2,2020/02/28,2019-2020,...,BELMONT CRAGIN,0.0,1.090994e+08,0.0,19.0,19.0,0.0,0.0,43311.707379,0
1558,1528,22CP001810,2022/02/03,6654 S LOOMIS ST,POLICE,0.0,2022,2,2022/02/03,2021-2022,...,WEST PULLMAN,0.0,9.936520e+07,0.0,53.0,53.0,0.0,0.0,50023.843001,0
1562,54,20CP008001,2020/02/06,5112 S NASHVILLE,POLICE,500.0,2020,2,2020/02/06,2019-2020,...,GARFIELD RIDGE,0.0,1.178908e+08,0.0,56.0,56.0,0.0,0.0,60080.447970,500


In [20]:
df_police.to_csv("../results/police-fines.csv")

In [21]:
df_police_by_community = df_police.groupby('community').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()
df_police_by_community.sort_values(by="n_dockets",ascending=False).reset_index()

,index,community,sum_fine_amt,n_dockets,n_records
0,0,BELMONT CRAGIN,150.0,5,6
1,7,WEST ENGLEWOOD,50.0,5,6
2,3,ENGLEWOOD,300.0,4,5
3,4,GARFIELD RIDGE,1000.0,3,3
4,9,WOODLAWN,0.0,2,2
5,1,CHICAGO LAWN,0.0,1,1
6,2,CLEARING,0.0,1,1
7,5,GREATER GRAND CROSSING,150.0,1,2
8,6,MORGAN PARK,0.0,1,1
9,8,WEST PULLMAN,0.0,1,1


# 5a. Englewood Deep Dive (then Garfield Ridge, then West Englewood)

In [22]:
df_englewood = df_by_address[df_by_address['community']=='ENGLEWOOD']
df_englewood[df_englewood['n_dockets']>=2].sort_values("n_dockets",ascending=False)

,Violation Address,community,sum_fine_amt,n_dockets,n_records
1717,932 W 59TH ST,ENGLEWOOD,8500.0,4,10
1310,5757 S PEORIA ST,ENGLEWOOD,1000.0,2,2
1585,7059 S HALSTED ST,ENGLEWOOD,1050.0,2,4
